In [1]:
import pandas as pd

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Feature Engineering

### Описание датасета:

- `id`: идентификатор записи;
- `url`: URL-записи о продаже;
- `region`: регион;
- `region_url`: URL региона;
- `price`: стоимость;
- `year`: год выпуска;
- `manufacturer`: производитель;
- `model`: модель;
- `condition`: состояние;
- `cylinders`: количество цилиндров;
- `fuel`: тип топлива;
- `odometer`: количество пройденных миль;
- `title_status`: статус;
- `transmission`: коробка передач;
- `VIN`: идентификационный номер;
- `drive`: тип привода;
- `size`: размер;
- `type`: кузов;
- `paint_color`: цвет;
- `image_url`: URL изображения;
- `description`: указанное описание;
- `county`: страна;
- `state`: штат;
- `lat`: широта;
- `long`: долгота;
- `posting_date`: дата размещения объявления о продаже;
- `price_category`: категория цены.

### Построение признаков из строк

In [2]:
df = pd.read_csv('data/vehicles_dataset_for_fe.csv')
df.head()

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,...,is_jeep,is_honda,is_nissan,x0_diesel,x0_electric,x0_gas,x0_hybrid,x0_other,std_scaled_odometer,std_scaled_price
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020,ram,2500 crew cab big horn,diesel,27442,...,0,0,0,1.0,0.0,0.0,0.0,0.0,-1.079390,2.958509
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016,ford,explorer 4wd 4dr xlt,other,60023,...,0,0,0,0.0,0.0,0.0,0.0,1.0,-0.560115,-0.085826
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017,volkswagen,golf r hatchback,gas,14048,...,0,0,0,0.0,0.0,1.0,0.0,0.0,-1.292863,1.406256
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013,toyota,rav4,gas,117291,...,0,0,0,0.0,0.0,1.0,0.0,0.0,0.352621,-0.281218
4,7303797340,https://knoxville.craigslist.org/ctd/d/knoxvil...,knoxville,https://knoxville.craigslist.org,14590,2012,bmw,1 series 128i coupe 2d,other,80465,...,0,0,0,0.0,0.0,0.0,0.0,1.0,-0.234311,-0.274017


In [3]:
df.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'fuel', 'odometer', 'title_status', 'transmission',
       'image_url', 'description', 'state', 'lat', 'long', 'posting_date',
       'price_category', 'date', 'odometer_km', 'odometer/price', 'region_new',
       'region_corrected', 'manufacturer_model', 'desc_len', 'model_in_desc',
       'price_k$', 'age_category', 'model_len', 'model_word_count', 'is_audi',
       'is_ford', 'is_chevrolet', 'is_toyota', 'is_jeep', 'is_honda',
       'is_nissan', 'x0_diesel', 'x0_electric', 'x0_gas', 'x0_hybrid',
       'x0_other', 'std_scaled_odometer', 'std_scaled_price'],
      dtype='object')

Cписок значений и количество уникальных значений для модели автомобиля.

In [4]:
df.model.unique()

array(['2500 crew cab big horn', 'explorer 4wd 4dr xlt',
       'golf r hatchback', ..., 'gs350', '1988 Nisan',
       'a3 2.0t premium plus pzev'], dtype=object)

In [5]:
df.model.nunique()

3341

Количество уникальных значений велико. Попробуем их сократить, оставив только первое слово в наименовании модели. Cоздаlbv новый признак `short_model`, содержащий только первое слово из наименования модели, выведем его и количество получившихся уникальных значений.

In [6]:
df['short_model'] = df.model.apply(lambda x: x.split()[0].lower())
df['short_model']

0           2500
1       explorer
2           golf
3           rav4
4              1
          ...   
9614        rav4
9615    wrangler
9616          a3
9617     cayenne
9618        1500
Name: short_model, Length: 9619, dtype: object

In [7]:
df.short_model.nunique()

906

### Преобразование категориальных переменных

In [8]:
for elem in ['short_model', 'transmission', 'region', 'manufacturer', 'state', 'title_status', 'age_category']:
    print(f'Наименование категории: \"{elem}\", количество уникальных значений в ней: {df[elem].nunique()}')

Наименование категории: "short_model", количество уникальных значений в ней: 906
Наименование категории: "transmission", количество уникальных значений в ней: 3
Наименование категории: "region", количество уникальных значений в ней: 393
Наименование категории: "manufacturer", количество уникальных значений в ней: 40
Наименование категории: "state", количество уникальных значений в ней: 51
Наименование категории: "title_status", количество уникальных значений в ней: 6
Наименование категории: "age_category", количество уникальных значений в ней: 3


Датафрейм `data`, содержащий только выбранные категориальные переменные.

In [9]:
data = df[['short_model', 'transmission', 'region', 'manufacturer', 'state', 'title_status', 'age_category']]
data

,short_model,transmission,region,manufacturer,state,title_status,age_category
0,2500,other,chattanooga,ram,tn,clean,new
1,explorer,automatic,north jersey,ford,nj,clean,new
2,golf,other,reno / tahoe,volkswagen,ca,clean,new
3,rav4,automatic,fayetteville,toyota,nc,clean,average
4,1,other,knoxville,bmw,tn,clean,average
...,...,...,...,...,...,...,...
9614,rav4,automatic,chautauqua,toyota,ny,clean,old
9615,wrangler,other,binghamton,jeep,ny,clean,average
9616,a3,automatic,salem,audi,or,clean,average
9617,cayenne,automatic,madison,porsche,wi,clean,new


In [10]:
ohe = OneHotEncoder(sparse=False)
ohe.fit(data)

OneHotEncoder(sparse=False)

In [11]:
ohe_categories = ohe.transform(data)
ohe_categories.shape

(9619, 1402)

Новые наименования признаков

In [12]:
ohe.get_feature_names()

array(['x0_-benz', 'x0_1', 'x0_124', ..., 'x6_average', 'x6_new',
       'x6_old'], dtype=object)

Добавим в исходный датафрейм получившиеся новые признаки

In [13]:
df[ohe.get_feature_names()] = ohe_categories
df.head()

C:\anaconda\lib\site-packages\pandas\core\frame.py:3678: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,...,x4_wy,x5_clean,x5_lien,x5_missing,x5_parts only,x5_rebuilt,x5_salvage,x6_average,x6_new,x6_old
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020,ram,2500 crew cab big horn,diesel,27442,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016,ford,explorer 4wd 4dr xlt,other,60023,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017,volkswagen,golf r hatchback,gas,14048,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013,toyota,rav4,gas,117291,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,7303797340,https://knoxville.craigslist.org/ctd/d/knoxvil...,knoxville,https://knoxville.craigslist.org,14590,2012,bmw,1 series 128i coupe 2d,other,80465,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [14]:
df.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'fuel', 'odometer',
       ...
       'x4_wy', 'x5_clean', 'x5_lien', 'x5_missing', 'x5_parts only',
       'x5_rebuilt', 'x5_salvage', 'x6_average', 'x6_new', 'x6_old'],
      dtype='object', length=1446)

### Стандартизация данных

Создадим переменную под количественные переменные `lat`, `long`, `year`, `odometer/price`, `desc_len`, `model_in_desc`, `model_len`, `model_word_count` и создадим датафрейм `data`, содержащий только выбранные количественные переменные.

In [15]:
data = df[['lat', 'long', 'year', 'odometer/price', 'desc_len', 'model_in_desc', 'model_len', 'model_word_count']]
data

,lat,long,year,odometer/price,desc_len,model_in_desc,model_len,model_word_count
0,35.060000,-85.250000,2020,0.499036,4482,0,22,5
1,40.821805,-74.061962,2016,3.542852,968,0,20,4
2,33.779214,-84.411811,2017,0.394718,4286,0,16,3
3,35.715954,-78.655304,2013,8.089034,3241,0,4,1
4,35.970000,-83.940000,2012,5.515079,4851,0,22,5
...,...,...,...,...,...,...,...,...
9614,42.123900,-79.189500,2002,33.370412,1710,0,4,1
9615,43.216990,-77.755610,2008,7.835323,948,0,8,1
9616,44.925908,-122.982753,2011,16.696387,909,0,25,5
9617,43.029559,-89.397796,2015,1.941787,3644,0,7,1


2. Создайте объект `std_scaler` класса StandardScaler с параметрами по умолчанию, который будет отвечать за стандартизацию.

In [16]:
std_scaler = StandardScaler()
std_scaler.fit(data)
std_scaled = std_scaler.transform(data)

Добавим в исходный датафрейм получившиеся новые признаки

In [17]:
df[['lat_std', 'long_std', 'year_std', 'odometer/price_std', 'desc_len_std', 'model_in_desc_std', 'model_len_std', 'model_word_count_std']] = std_scaled
df.head()

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,...,x6_new,x6_old,lat_std,long_std,year_std,odometer/price_std,desc_len_std,model_in_desc_std,model_len_std,model_word_count_std
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020,ram,2500 crew cab big horn,diesel,27442,...,1.0,0.0,-0.619172,0.484245,1.322394,-0.510784,0.632075,-0.155788,1.163032,1.910669
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016,ford,explorer 4wd 4dr xlt,other,60023,...,1.0,0.0,0.388014,1.110800,0.695973,-0.402947,-0.646781,-0.155788,0.932087,1.235799
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017,volkswagen,golf r hatchback,gas,14048,...,1.0,0.0,-0.843059,0.531185,0.852578,-0.514480,0.560744,-0.155788,0.470197,0.560930
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013,toyota,rav4,gas,117291,...,0.0,0.0,-0.504509,0.853562,0.226157,-0.241883,0.180435,-0.155788,-0.915473,-0.788810
4,7303797340,https://knoxville.craigslist.org/ctd/d/knoxvil...,knoxville,https://knoxville.craigslist.org,14590,2012,bmw,1 series 128i coupe 2d,other,80465,...,0.0,0.0,-0.460101,0.557607,0.069552,-0.333074,0.766366,-0.155788,1.163032,1.910669


In [18]:
df.columns

Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'fuel', 'odometer',
       ...
       'x6_new', 'x6_old', 'lat_std', 'long_std', 'year_std',
       'odometer/price_std', 'desc_len_std', 'model_in_desc_std',
       'model_len_std', 'model_word_count_std'],
      dtype='object', length=1454)

### Признаки на основе дат

In [19]:
df.posting_date

0       2021-04-17T12:30:50-0400
1       2021-05-03T15:40:21-0400
2       2021-04-28T03:52:20-0700
3       2021-04-17T10:08:57-0400
4       2021-04-08T15:10:56-0400
                  ...           
9614    2021-04-10T16:33:57-0400
9615    2021-05-03T09:36:30-0400
9616    2021-04-22T12:14:01-0700
9617    2021-04-14T09:14:42-0500
9618    2021-04-24T13:50:49-0400
Name: posting_date, Length: 9619, dtype: object

In [20]:
df.date

0       2021-04-17 16:30:50+00:00
1       2021-05-03 19:40:21+00:00
2       2021-04-28 10:52:20+00:00
3       2021-04-17 14:08:57+00:00
4       2021-04-08 19:10:56+00:00
                  ...            
9614    2021-04-10 20:33:57+00:00
9615    2021-05-03 13:36:30+00:00
9616    2021-04-22 19:14:01+00:00
9617    2021-04-14 14:14:42+00:00
9618    2021-04-24 17:50:49+00:00
Name: date, Length: 9619, dtype: object

Колонка `date` содержит данные в строковом формате, поэтому изменим её тип 

In [21]:
df.date = pd.to_datetime(df.date, utc=True)
df.date

0      2021-04-17 16:30:50+00:00
1      2021-05-03 19:40:21+00:00
2      2021-04-28 10:52:20+00:00
3      2021-04-17 14:08:57+00:00
4      2021-04-08 19:10:56+00:00
                  ...           
9614   2021-04-10 20:33:57+00:00
9615   2021-05-03 13:36:30+00:00
9616   2021-04-22 19:14:01+00:00
9617   2021-04-14 14:14:42+00:00
9618   2021-04-24 17:50:49+00:00
Name: date, Length: 9619, dtype: datetime64[ns, UTC]

Создадим признак `month`, обозначающий номер месяца размещения объявления о продаже автомобиля

In [22]:
df['month'] = df.date.dt.month
df[['date', 'month']]

,date,month
0,2021-04-17 16:30:50+00:00,4
1,2021-05-03 19:40:21+00:00,5
2,2021-04-28 10:52:20+00:00,4
3,2021-04-17 14:08:57+00:00,4
4,2021-04-08 19:10:56+00:00,4
...,...,...
9614,2021-04-10 20:33:57+00:00,4
9615,2021-05-03 13:36:30+00:00,5
9616,2021-04-22 19:14:01+00:00,4
9617,2021-04-14 14:14:42+00:00,4


Создадим признак `dayofweek`, обозначающий день недели размещения объявления о продаже автомобиля

In [23]:
df.posting_date = pd.to_datetime(df.posting_date, utc=True)
df.posting_date

0      2021-04-17 16:30:50+00:00
1      2021-05-03 19:40:21+00:00
2      2021-04-28 10:52:20+00:00
3      2021-04-17 14:08:57+00:00
4      2021-04-08 19:10:56+00:00
                  ...           
9614   2021-04-10 20:33:57+00:00
9615   2021-05-03 13:36:30+00:00
9616   2021-04-22 19:14:01+00:00
9617   2021-04-14 14:14:42+00:00
9618   2021-04-24 17:50:49+00:00
Name: posting_date, Length: 9619, dtype: datetime64[ns, UTC]

In [24]:
df['dayofweek'] = df.posting_date.dt.day_name()
df[['posting_date', 'dayofweek']]

,posting_date,dayofweek
0,2021-04-17 16:30:50+00:00,Saturday
1,2021-05-03 19:40:21+00:00,Monday
2,2021-04-28 10:52:20+00:00,Wednesday
3,2021-04-17 14:08:57+00:00,Saturday
4,2021-04-08 19:10:56+00:00,Thursday
...,...,...
9614,2021-04-10 20:33:57+00:00,Saturday
9615,2021-05-03 13:36:30+00:00,Monday
9616,2021-04-22 19:14:01+00:00,Thursday
9617,2021-04-14 14:14:42+00:00,Wednesday


Создадим признак `diff_years`, обозначающий количество лет между годом производства автомобиля и годом размещения объявления о продаже автомобиля

In [25]:
df['diff_years'] = df.apply(lambda x: x.posting_date.year - x.year, axis=1)
df[['year', 'posting_date', 'diff_years']]

,year,posting_date,diff_years
0,2020,2021-04-17 16:30:50+00:00,1
1,2016,2021-05-03 19:40:21+00:00,5
2,2017,2021-04-28 10:52:20+00:00,4
3,2013,2021-04-17 14:08:57+00:00,8
4,2012,2021-04-08 19:10:56+00:00,9
...,...,...,...
9614,2002,2021-04-10 20:33:57+00:00,19
9615,2008,2021-05-03 13:36:30+00:00,13
9616,2011,2021-04-22 19:14:01+00:00,10
9617,2015,2021-04-14 14:14:42+00:00,6


Применим стандартизацию к новым признаками и сохраним результат в новые признаки

In [26]:
std_scaler.fit(df[['month', 'diff_years']])
std_scaled = std_scaler.transform(df[['month', 'diff_years']])
df[['month_std', 'diff_years_std']] = std_scaled

In [27]:
ohe.fit(df[['dayofweek']])
ohe_dayofweek = ohe.transform(df[['dayofweek']])
df[ohe.get_feature_names()] = ohe_dayofweek

### Сохранение датафрейма для этапа моделирования

Удалим колонки, которые были исходными для формирования признаков. Сохраним новый датафрейм в переменную `df_prepared`.

In [28]:
columns_for_drop = ['year', 'url', 'region', 'region_url', 'manufacturer',
                    'model', 'fuel', 'odometer', 'title_status', 'transmission',
                    'image_url', 'description', 'state', 'lat', 'long', 'posting_date',
                    'odometer_km', 'odometer/price', 'region_new', 'region_corrected', 'manufacturer_model',
                    'desc_len', 'model_in_desc', 'price_k$', 'age_category', 'model_len', 'model_word_count',
                    'short_model', 'date', 'std_scaled_price',
                    'month', 'dayofweek', 'diff_years',
                    'odometer/price_std']
df_prepared = df.drop(columns_for_drop, axis=1)

Оставшийся список колонок и размерность финального датафрейма.

In [29]:
df_prepared.columns

Index(['id', 'price', 'price_category', 'is_audi', 'is_ford', 'is_chevrolet',
       'is_toyota', 'is_jeep', 'is_honda', 'is_nissan',
       ...
       'model_word_count_std', 'month_std', 'diff_years_std', 'x0_Friday',
       'x0_Monday', 'x0_Saturday', 'x0_Sunday', 'x0_Thursday', 'x0_Tuesday',
       'x0_Wednesday'],
      dtype='object', length=1432)

In [30]:
df_prepared.shape

(9619, 1432)

### Описание преобразованного датасета:

- `id`: идентификатор записи;
- `is_manufacturer_name`: признак производителя автомобиля;
- `region_*`: регион;
- `x0_*`: тип топлива;
- `manufacturer_*`: производитель;
- `short_model_*`: сокращённая модель автомобиля;
- `title_status_*`: статус;
- `transmission_*`: коробка передач;
- `state_*`: штат;
- `age_category_*`: возрастная категория автомобиля;
- `std_scaled_odometer`: количество пройденных миль (после стандартизации);
- `year_std`: год выпуска (после стандартизации);
- `lat_std`: широта (после стандартизации);
- `long_std`: долгота (после стандартизации);
- `odometer/price_std`: отношение стоимости к пробегу автомобиля (после стандартизации);
- `desc_len_std`: количество символов в тексте объявления о продаже (после стандартизации);
- `model_in_desc_std`: количество наименований модели автомобиля в тексте объявления о продаже (после стандартизации);
- `model_len_std`: длина наименования автомобиля (после стандартизации);
- `model_word_count_std`: количество слов в наименовании автомобиля (после стандартизации);
- `month_std`: номер месяца размещения объявления о продаже автомобиля (после стандартизации);
- `dayofweek_std`: день недели размещения объявления о продаже автомобиля (после стандартизации);
- `diff_years_std`: количество лет между годом производства автомобиля и годом размещения объявления о продаже автомобиля (после стандартизации);
- `price`: стоимость;
- `price_category`: категория цены.